In [11]:
import streamlit as st
import pandas as pd
import easyocr as ocr
from PIL import Image
import numpy as np
import sqlite3

db = sqlite3.connect("OCR.db")
cur = db.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS Data (id INTEGER PRIMARY KEY AUTOINCREMENT, image BLOB, text_data VARCHAR(255))")

st.title("Business Card Extraction using OCR")
img = st.file_uploader(label="Upload Your Image", type=("jpg", "png", "jpeg"), accept_multiple_files=True)

if img:
    @st.cache_data
    def load():
        reader = ocr.Reader(['en'], gpu=True)
        return reader

    reader = load()

    if img is not None:
        for i in img:
            input = Image.open(i)
            st.image(i, caption='Uploaded Image', use_column_width='auto')
            res = reader.readtext(np.array(input))
            st.subheader("Extracted Data")
            data = []
            for text in res:
                data.append({'Text': text[1]})
            df = pd.DataFrame(data)
            st.experimental_show(df)
            st.success("Successfully extracted the text data from the business card image", icon="✅")

            store = st.button("Click here to save the data")
            if store:
                try:
                    ins_query = "INSERT INTO Data (image, text_data) VALUES (?, ?)"
                    val = (sqlite3.Binary(input.tobytes()), text[1])
                    cur.execute(ins_query, val)
                    st.info("Data stored successfully")
                except Exception as e:
                    st.error(f"An error occurred while storing the data: {e}")

            update = st.button("Click here to update the data")
            if update:
                row_id = st.text_input("Enter the ID of the row to be updated:")
                new_text_data = st.text_input("Enter the new value of the text_data column:")
                cur.execute("UPDATE Data SET text_data = ? WHERE id = ?", (new_text_data, row_id))
                st.info("Updated successfully")

            delete = st.button("Click here to delete the data stored")
            if delete:
                cur.execute("DELETE FROM Data")
                st.info("Deleted successfully")
